# 1. Preprocessing

## 1.1. Imports

In [102]:
# Imports

import pandas as pd
import numpy as np
import string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1.2. Preprocessing Functions

In [105]:
# Load data

def get_data(nrows=500_000):
    '''returns a DataFrame with nrows from downloaded Keggle csv in raw_data folder'''
    dataset_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_1.csv", nrows=nrows) 
    df = dataset_1.copy()
    return df


In [106]:
# Clean data

def clean_data(df):
    '''returns cleaned DataFrame'''
    
    # dropping redundant columns
    df = df[['Negative_Review', 'Positive_Review', 'Reviewer_Score']]

    # Cleaning, merging and renaming negative and positive reviews
    df[['Negative_Review']] = df[['Negative_Review']].replace(to_replace="No Negative", value="")
    df[['Positive_Review']] = df[['Positive_Review']].replace(to_replace="No Positive", value="")
    df["reviews"] = df['Negative_Review'] + " " + df['Positive_Review']
    df["review_score"] = df['Reviewer_Score']
    df = df.drop(columns=['Negative_Review', 'Positive_Review', 'Reviewer_Score'])

    # Remove reviews with less than 6 words (or signs)
    df['length'] = df['reviews'].apply(lambda x: len(word_tokenize(str(x))))
    df.drop(df[df['length'] < 6].index, inplace=True)
    df.drop(columns=['length'], inplace=True)
    df.reset_index(drop=True, inplace=True)

    return df

In [107]:
# Balance data

def balance_data(df):
  df_1 = df[df['review_score'] < 5][:10000]
  df_4 = df[(df['review_score'] > 9) & (df['review_score'] < 10.1)][:10000]
  df = pd.concat([df_1,df_4])

  return df

In [108]:
# Clean for NLP

def custom_stopwords():
    """create custom stopwords list excluding negative words"""
    negative_words = ['no',
    'nor',
    'not',
    "don't",
    'should',
    "should've",
    'aren',
    "aren't",
    'couldn',
    "couldn't",
    'didn',
    "didn't",
    'doesn',
    "doesn't",
    'hadn',
    "hadn't",
    'hasn',
    "hasn't",
    'haven',
    "haven't",
    'isn',
    "isn't",
    "wasn't",
    'weren',
    "weren't",
    'won',
    "won't",
    'wouldn',
    "wouldn't"]

    custom_stopwords = [x for x in stopwords.words('english') if x not in negative_words]

    extra_stopwords = ["hotel","everything","anything","thing"]  #customize extra stop_words

    custom_stopwords.extend(extra_stopwords)

    return custom_stopwords


def clean_for_nlp(text):
    """ preprocess review text data for nlp analysis """
    # Lower case
    text = ''.join(text)
    text = text.lower()
    # Remove numbers
    text = ''.join(word for word in text if not word.isdigit())
    # Remove punctuation
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    # Remove stopwords
    text = word_tokenize(text)
    stopwords = custom_stopwords()
    text = [w for w in text if not w in stopwords]
    # Lemmatizing
    lemmatizer = WordNetLemmatizer()
    text = [lemmatizer.lemmatize(word) for word in text]
    text = ' '.join(word for word in text)

    return(text)

In [109]:
# preprocess df for NN

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def tokenInit(train, max_words=5000):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train)

    return tokenizer


def padding(X):
  tokenizer = tokenInit(X)
  sequences = tokenizer.texts_to_sequences(X)
  X_pad = pad_sequences(sequences, dtype='int32', padding='post')

  return X_pad

## 1.3. Preprocessing Data (applying preprocessing functions to df)

In [110]:
# get data

df = get_data()
df = clean_data(df)
df = balance_data(df)
df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

,reviews,review_score
0,I am so angry that i made this post available...,2.9
3,My room was dirty and I was afraid to walk ba...,3.8
6,Cleaner did not change our sheet and duvet ev...,4.6
12,The floor in my room was filfy dirty Very bas...,4.6
21,Our room was an overrated disaster room 231 d...,3.8
...,...,...
20095,Definitely above expectations experience base...,9.6
20099,Would have liked tea coffee making in room wa...,10.0
20101,Clean comfortable and excellent service Noth...,10.0
20109,Normal rooms are a bit small but that is Pari...,10.0


In [111]:
# Define X and y

X = df[['reviews']]
y = df[['review_score']]

# check
print(X.shape, y.shape)

(20000, 1) (20000, 1)


# 2. Model

 ## 2.1. Hold out

In [112]:
from sklearn.model_selection import train_test_split

# Hold out 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#check
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((16000, 1), (4000, 1), (16000, 1), (4000, 1))

 ## 2.2. NN

### 2.2.1. Model Architecture

In [113]:
from sklearn.model_selection import cross_validate
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import Sequential
from tensorflow.keras import regularizers


def initialize_model():
    ### Model architecture
    model = models.Sequential()
 
    ### Embedding Padded
    model.add(layers.Embedding(input_dim=5000, output_dim=100, mask_zero=True))
        
    ### First convolution & max-pooling
    model.add(layers.LSTM(units=100, activation='tanh', return_sequences=True))
    model.add(layers.LSTM(units=100, activation='tanh', return_sequences=True))
    model.add(layers.LSTM(units=50, activation='tanh'))
    model.add(layers.Dropout(0.2))                     #change params
    model.add(layers.Dense(40, activation='relu', kernel_regularizer=regularizers.L1(0.01)))    #Use regulazers
    model.add(layers.Dropout(0.2))                     #change params
    model.add(layers.Dense(20, activation='relu', kernel_regularizer=regularizers.L1(0.01)))    #Use regulazers
    model.add(layers.Dropout(0.2))                     #change params
    model.add(layers.Dense(10, activation='relu', kernel_regularizer=regularizers.L1(0.01)))    #Use regulazers
    model.add(layers.Dropout(0.2))                     #change params 

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(1, activation='linear'))
        
    ### Model compilation
    model.compile(loss='mse', 
                  optimizer='rmsprop',
                  metrics=['mae'])     

    return model

### 2.2.2. Make Model Pickleable

In [114]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils


def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

# Hotfix function
def make_keras_picklable():

    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))

    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

### 2.2.3. Initialize Model

In [115]:
# initialize model (actually not necessary here, just to overview summary)

model = initialize_model()
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, None, 100)         500000    
_________________________________________________________________
lstm_33 (LSTM)               (None, None, 100)         80400     
_________________________________________________________________
lstm_34 (LSTM)               (None, None, 100)         80400     
_________________________________________________________________
lstm_35 (LSTM)               (None, 50)                30200     
_________________________________________________________________
dropout_44 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_44 (Dense)             (None, 40)                2040      
_________________________________________________________________
dropout_45 (Dropout)         (None, 40)              

# 3. Pipelining, Fitting and Exporting Model

## 3.1. Built Wrapper for Keras Model (to save it into a .joblib format)

In [116]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

nn_model = KerasRegressor(build_fn = initialize_model)

## 3.2. Custom Transformer for Pipeline

In [117]:
from sklearn.base import BaseEstimator, TransformerMixin

class TextProcessor(BaseEstimator, TransformerMixin):
  """ Custom Transformer for cleaning and preprocessing string into required format for NN model """
  
  def __init__(self, max_words=5000):
    self.tokenizer = Tokenizer(num_words=max_words)
  
  def fit(self, X, y=None):
    # cleaning text
    X = list(map(clean_for_nlp, X['reviews']))
    self.tokenizer.fit_on_texts(X)
    return self

  def transform(self, X, y=None):
    # cleaning text
    X = list(map(clean_for_nlp, X['reviews']))
    # tokenizing
    sequences = self.tokenizer.texts_to_sequences(X)
    # padding
    X = pad_sequences(sequences, dtype='int32', padding='post')

    return X


## 3.3. Build Pipeline

In [118]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

def set_pipeline():
  """defines the pipeline"""
  preproc_pipe = Pipeline([('text_preprocessor', TextProcessor())])

  pipeline = Pipeline([('preproc_pipe', preproc_pipe), ('nn_model', nn_model)])

  return pipeline

In [119]:
# Set pipeline and initialize model
pipeline = set_pipeline()

## 3.4. Fit Pipeline

In [120]:
# Fitting
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

pipeline.fit(X_train, y_train,
          nn_model__validation_split=0.2,
          nn_model__batch_size=32,
          nn_model__epochs=200,
          nn_model__verbose=1,
          nn_model__callbacks=[es])

Epoch 1/200
400/400 [==============================] - 20s 34ms/step - loss: 22.0121 - mae: 3.4850 - val_loss: 4.6396 - val_mae: 1.1304
Epoch 2/200
400/400 [==============================] - 11s 28ms/step - loss: 7.9244 - mae: 1.8430 - val_loss: 3.5125 - val_mae: 0.8798
Epoch 3/200
400/400 [==============================] - 11s 28ms/step - loss: 6.0561 - mae: 1.5665 - val_loss: 3.6758 - val_mae: 1.0746
Epoch 4/200
400/400 [==============================] - 11s 27ms/step - loss: 5.2487 - mae: 1.4506 - val_loss: 3.5867 - val_mae: 1.1403
Epoch 5/200
400/400 [==============================] - 12s 29ms/step - loss: 4.4807 - mae: 1.3519 - val_loss: 2.8382 - val_mae: 0.8503
Epoch 6/200
400/400 [==============================] - 11s 28ms/step - loss: 4.2329 - mae: 1.3382 - val_loss: 2.5895 - val_mae: 0.7870
Epoch 7/200
400/400 [==============================] - 11s 28ms/step - loss: 3.8506 - mae: 1.2770 - val_loss: 2.5529 - val_mae: 0.7661
Epoch 8/200
400/400 [==============================] -

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('preproc_pipe',
                 Pipeline(memory=None,
                          steps=[('text_preprocessor',
                                  TextProcessor(max_words=None))],
                          verbose=False)),
                ('nn_model',
                 <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fb5f0363790>)],
         verbose=False)

## 3.5. Evaluate Model

In [136]:
pipeline.score(X_test,y_test)

125/125 [==============================] - 1s 11ms/step - loss: 2.5166 - mae: 0.8812


## 3.6. Export to .joblib Format

In [121]:
import joblib
from termcolor import colored

def save_model(pipeline):
  """Save the model into a .joblib format"""
  joblib.dump(pipeline, 'model.joblib')
  print(colored("model.joblib saved locally", "green"))

In [122]:
save_model(pipeline)

model.joblib saved locally


In [123]:
pipeline_test = joblib.load('model.joblib')

## 3.7. Test

In [124]:
Z = pd.DataFrame({"reviews": ["This hotel was very nice, I loved it. Breakfast was amazing!"]})

In [125]:
result = pipeline_test.predict(Z)
result

array(9.147982, dtype=float32)